# Imports

In [1]:
from dotenv import load_dotenv
import logging

load_dotenv(dotenv_path=".env.evaluation", verbose=True, override=True)
logging.basicConfig(level=logging.DEBUG)

In [2]:
import os

from autocommit_evaluation.core.enums import EnvironmentKey
from autocommit_evaluation.cmg.evaluators import CommitMessageGenerator
from autocommit_evaluation.cmg import evaluator
from autocommit_evaluation.core import (
    open_ai_few_shot_high_level_context_cmg_chain,
    open_ai_low_level_context_cmg_chain,
    open_ai_zero_shot_high_level_context_cmg_chain,
    open_ai_high_level_context_chain,
    openrouter_zero_shot_high_level_context_cmg_chain,
    openrouter_high_level_context_chain,
)
from autocommit.core.models import CommitDataModel
from autocommit_evaluation.datapreparation import context_generator, example_generator

# Initialization

In [3]:
COMMIT_DATA_JSON_FILE_PATH = os.path.join("autocommit_evaluation", "data", "cmg", "commits.evaluation.json")
CONTEXT_DATA_PATH = os.path.join("autocommit_evaluation", "data", "context")

DEFAULT_CONTEXT_GENERATION_OUTPUT_PATH = os.path.join(
    "autocommit_evaluation", "data", "context"
)
DEFAULT_HIGH_LEVEL_CONTEXT_OUTPUT_PATH = os.path.join(
    "out", "evaluation", "highlevelcontext"
)
DEFAULT_CMG_OUTPUT_PATH = os.path.join("out", "evaluation", "cmg")
DEFAULT_DIFF_CLASSIFICATION_OUTPUT_PATH = os.path.join(
    "out", "evaluation", "diffclassification"
)

DIFF_CLASSIFIER_CHAINS = [
    open_ai_zero_shot_high_level_context_cmg_chain,
    open_ai_low_level_context_cmg_chain,
    openrouter_zero_shot_high_level_context_cmg_chain
]

HIGH_LEVEL_CONTEXT_CHAINS = [
    open_ai_high_level_context_chain,
    openrouter_high_level_context_chain,
]

GENERATORS = [
    CommitMessageGenerator(
        "Open AI Zero-Shot High-Level Context Generator", open_ai_zero_shot_high_level_context_cmg_chain
    ),
    CommitMessageGenerator(
        "Open AI Few-Shot High-Level Context Generator", open_ai_few_shot_high_level_context_cmg_chain
    ),
    CommitMessageGenerator("Open AI Low-Level Context Generator", open_ai_low_level_context_cmg_chain),
    CommitMessageGenerator(
        "OpenRouter Zero-Shot High-Level Context Generator", openrouter_zero_shot_high_level_context_cmg_chain
    )
]

In [4]:
CONTEXT_GENERATION_OUTPUT_PATH = os.getenv(
        EnvironmentKey.CONTEXT_GENERATION_OUTPUT_PATH.value,
        DEFAULT_CONTEXT_GENERATION_OUTPUT_PATH,
    )

HIGH_LEVEL_CONTEXT_OUTPUT_PATH = os.getenv(
        EnvironmentKey.HIGH_LEVEL_CONTEXT_OUTPUT_PATH.value,
        DEFAULT_HIGH_LEVEL_CONTEXT_OUTPUT_PATH,
    )

CMG_OUTPUT_PATH = os.getenv(
        EnvironmentKey.CMG_OUTPUT_PATH.value, DEFAULT_CMG_OUTPUT_PATH
    )

DIFF_CLASSIFICATION_OUTPUT_PATH = os.getenv(
        EnvironmentKey.DIFF_CLASSIFICATION_OUTPUT_PATH.value,
        DEFAULT_DIFF_CLASSIFICATION_OUTPUT_PATH,
    )

In [5]:
def get_commits(path: str) -> list[CommitDataModel]:
        with open(path, "r", encoding="utf-8") as file:
            json_string = file.read()

        return CommitDataModel.from_json(json_string)

COMMITS = get_commits(COMMIT_DATA_JSON_FILE_PATH)

# Classify Diff

In [6]:
# INCLUDED_DIFF_CLASSIFIER_CHAIN_INDEXES = [0, 1, 2]

# for index in INCLUDED_DIFF_CLASSIFIER_CHAIN_INDEXES:
#     evaluator.classify_diffs(DIFF_CLASSIFIER_CHAINS[index], COMMITS, CONTEXT_DATA_PATH, DIFF_CLASSIFICATION_OUTPUT_PATH)

# Generate Context

In [7]:
# context_generator.generate_context(COMMITS, CONTEXT_GENERATION_OUTPUT_PATH)

# Get High Level Context

In [8]:
# INCLUDED_HIGH_LEVEL_CONTEXT_CHAIN_INDEXES = [0, 1]

# for index in INCLUDED_HIGH_LEVEL_CONTEXT_CHAIN_INDEXES:
#     evaluator.get_high_level_contexts(
#         HIGH_LEVEL_CONTEXT_CHAINS[index],
#         COMMITS, 
#         CONTEXT_DATA_PATH, 
#         HIGH_LEVEL_CONTEXT_OUTPUT_PATH)

# Generate Commit Message

In [9]:
INCLUDED_GENERATOR_INDEXES = [0, 1, 2, 3]
INCLUDED_GENERATOR_INDEXES = [3]

filtered_generators = [GENERATORS[i] for i in INCLUDED_GENERATOR_INDEXES]
evaluator.evaluate(filtered_generators, COMMITS, CONTEXT_DATA_PATH, CMG_OUTPUT_PATH)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.smith.langchain.com:443
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'Given a Git diff and the relevant source code, write a concise summary of the code changes in a way that a non-technical person can understand. The query text must summarize the code changes in two very brief sentences.\n\nAvoid including any reasoning steps to the output.\n\nGit diff:\ndiff --git a/contrib/storage-hive/core/src/main/java/org/apache/drill/exec/store/hive/HiveStoragePlugin.java b/contrib/storage-hive/core/src/main/java/org/apache/drill/exec/store/hive/HiveStoragePlugin.java\nindex c021ebca1..1ce138c0e 100644\n--- a/contrib/storage-hive/core/src/main/java/org/apache/drill/exec/store/hive/HiveStoragePlugin.java\n+++ b/contrib/storage-hive/core/src/main/java/org/apache/drill/exec/store/hive/HiveStoragePlugin.java\n@@ -192,7 +192,7 @@ public

DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "POST /runs/multipart HTTP/11" 202 34
